In [ ]:
!pip install langchain langchain_community langchain_openai pypdf langsmith qdrant-client ragas pandas

In [2]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [3]:
import pandas as pd

test_df = pd.read_csv("synthetic_midterm_question_dataset.csv")

In [4]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationBufferMemory

In [6]:
pdf_paths = ["/Users/xico/AIMakerSpace-Midterm/AI_Risk_Management_Framework.pdf",
"/Users/xico/AIMakerSpace-Midterm/Blueprint-for-an-AI-Bill-of-Rights.pdf"]

In [7]:
pdf_documents = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_documents.extend(loader.load())

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
    )
pdf_docs = text_splitter.split_documents(pdf_documents)

In [11]:
baseline_metrics = pd.read_csv("medium_chunk_metrics.csv")
baseline_metrics.rename(columns={'MediumChunk': 'Baseline'}, inplace=True)
baseline_metrics


,Metric,Baseline
0,faithfulness,0.895359
1,answer_relevancy,0.955419
2,context_recall,0.934028
3,context_precision,0.937500
4,answer_correctness,0.629267


In [ ]:
pip install sentence-transformers

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("XicoC/midterm-finetuned-arctic")

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="XicoC/midterm-finetuned-arctic")

In [22]:
vectorstore = Qdrant.from_documents(
    documents=pdf_docs,
    embedding=embedding,
    location=":memory:",
    collection_name="Midterm Embedding Eval"
)

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10},
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
multiquery_ft_embedding_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=retriever_llm
)

In [24]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    streaming=True,
)

In [25]:
custom_template = """
You are an expert in artificial intelligence policy, ethics, and industry trends. Your task is to provide clear and accurate answers to questions related to AI's role in politics, government regulations, and its ethical implications for enterprises. Use reliable and up-to-date information from government documents, industry reports, and academic research to inform your responses. Make sure to consider how AI is evolving, especially in relation to the current political landscape, and provide answers in a way that is easy to understand for both AI professionals and non-experts.

Remember these key points:
1. Use "you" when addressing the user and "I" when referring to yourself.
2. If you encounter complex or legal language in the context, simplify it for easy understanding. Imagine you're explaining it to someone who isn't familiar with legal terms.
3. Be prepared for follow-up questions and maintain context from previous exchanges.
4. If there's no information from a retrieved document in the context to answer a question or if there are no documents to cite, say: "I'm sorry, I don't know the answer to that question."
5. When providing information, always cite the source document and page number in parentheses at the end of the relevant sentence or paragraph, like this: (Source: [document name], p. [page number]).

Here are a few example questions you might receive:

How are governments regulating AI, and what new policies have been implemented?
What are the ethical risks of using AI in political decision-making?
How can enterprises ensure their AI applications meet government ethical standards?

One final rule for you to remember. You CANNOT under any circumstance, answer any question that does not pertain to the AI. If you do answer an out-of-scope question, you could lose your job. If you are asked a question that does not have to do with AI, you must say: "I'm sorry, I don't know the answer to that question."
Context: {context}
Chat History: {chat_history}
Human: {question}
AI:"""

PROMPT = PromptTemplate(
    template=custom_template, input_variables=["context", "question", "chat_history"]
)

In [27]:
multiquery_ft_embedding_rag_chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=multiquery_ft_embedding_retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT},
        return_source_documents=True,
    )

In [28]:
multiquery_ft_embedding_rag_chain.invoke({"question": "What are Trustworthy AI Characteristics?"})

{'question': 'What are Trustworthy AI Characteristics?',
 'chat_history': [HumanMessage(content='What are Trustworthy AI Characteristics?'),
  AIMessage(content='Trustworthy AI characteristics refer to the essential qualities that artificial intelligence systems should possess to ensure they are reliable, ethical, and beneficial to society. Here are some key characteristics:\n\n1. **Accountable and Transparent**: AI systems should be designed in a way that their decision-making processes can be understood and scrutinized. This means providing clear explanations for how decisions are made and who is responsible for them.\n\n2. **Privacy Enhanced**: AI systems must prioritize user privacy and data protection. This involves implementing measures to safeguard personal information and ensuring that data is used responsibly.\n\n3. **Safe, Secure, and Resilient**: AI systems should be robust against attacks and failures. They must be designed to operate safely, even in unexpected situations, 

In [29]:
answers = []
contexts = []

for question in test_questions:
  response = multiquery_ft_embedding_rag_chain.invoke({"question" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["source_documents"]])

In [30]:
from datasets import Dataset

multiquery_ft_embedding_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [31]:
multiquery_ft_embedding_dataset[0]

{'question': 'What is the significance of providing notice and explanation as a legal requirement in the context of automated systems?',
 'answer': "Providing notice and explanation as a legal requirement in the context of automated systems is significant for several reasons:\n\n1. **Transparency**: It ensures that individuals are aware when automated systems are being used to make decisions that affect them. This transparency is crucial for building trust between the public and the entities deploying these systems.\n\n2. **Accountability**: By clearly identifying the entities responsible for the design and use of automated systems, it holds them accountable for the decisions made by these systems. This accountability is essential for addressing any potential harms or errors that may arise from automated decision-making.\n\n3. **Informed Decision-Making**: Notice and explanation allow individuals to understand how and why decisions are made, which is vital for them to contest or appeal

In [32]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [33]:
multiquery_ft_embedding_results = evaluate(multiquery_ft_embedding_dataset, metrics)

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

In [34]:
multiquery_ft_embedding_results

{'faithfulness': 0.8684, 'answer_relevancy': 0.9558, 'context_recall': 0.9444, 'context_precision': 0.9537, 'answer_correctness': 0.6034}

In [ ]:
multiquery_ft_embedding_results_df = multiquery_ft_embedding_results.to_pandas()
multiquery_ft_embedding_results_df.head()

In [36]:
multiquery_ft_embedding_results_df.to_csv("multiquery_ft_embedding_ragas_results.csv", index=False)

In [37]:
multiquery_ft_embedding_metrics_df = pd.DataFrame(list(multiquery_ft_embedding_results.items()), columns=['Metric', 'Fine-Tune Embedding'])

In [38]:
multiquery_ft_embedding_metrics_df

,Metric,Fine-Tune Embedding
0,faithfulness,0.868351
1,answer_relevancy,0.955777
2,context_recall,0.944444
3,context_precision,0.953668
4,answer_correctness,0.603407


In [39]:
multiquery_ft_embedding_metrics_df.to_csv("multiquery_ft_embedding_metrics.csv", index=False)

In [1]:
import pandas as pd

In [2]:
multiquery_metrics_df = pd.read_csv("multiquery_metrics.csv")

In [3]:
multiquery_ft_embedding_metrics_df = pd.read_csv("multiquery_ft_embedding_metrics.csv")

In [5]:
multiquery_metrics_df

,Metric,MultiQuery
0,faithfulness,0.896804
1,answer_relevancy,0.953211
2,context_recall,0.890625
3,context_precision,0.920732
4,answer_correctness,0.690058


In [6]:
df_baseline_ft_embeddings = pd.merge(multiquery_metrics_df, multiquery_ft_embedding_metrics_df, on='Metric')

df_baseline_ft_embeddings['Baseline -> Fine-Tune Embedding'] = df_baseline_ft_embeddings['Fine-Tune Embedding'] - df_baseline_ft_embeddings['MultiQuery']

df_baseline_ft_embeddings

,Metric,MultiQuery,Fine-Tune Embedding,Baseline -> Fine-Tune Embedding
0,faithfulness,0.896804,0.868351,-0.028452
1,answer_relevancy,0.953211,0.955777,0.002566
2,context_recall,0.890625,0.944444,0.053819
3,context_precision,0.920732,0.953668,0.032936
4,answer_correctness,0.690058,0.603407,-0.086651


In [41]:
df_baseline_ft_embeddings = pd.merge(baseline_metrics, multiquery_ft_embedding_metrics_df, on='Metric')

df_baseline_ft_embeddings['Baseline -> Fine-Tune Embedding'] = df_baseline_ft_embeddings['Fine-Tune Embedding'] - df_baseline_ft_embeddings['Baseline']

df_baseline_ft_embeddings

,Metric,Baseline,Fine-Tune Embedding,Baseline -> Fine-Tune Embedding
0,faithfulness,0.895359,0.868351,-0.027007
1,answer_relevancy,0.955419,0.955777,0.000358
2,context_recall,0.934028,0.944444,0.010417
3,context_precision,0.937500,0.953668,0.016168
4,answer_correctness,0.629267,0.603407,-0.025861
